In [97]:
import pandas as pd
import numpy as np
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, FunctionTransformer, MultiLabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LassoCV
from catboost import CatBoostRegressor

In [147]:
data = pd.read_excel('all_vacancies_clean.xlsx')
data.drop(columns=['id', 'name', 'alternate_url'], inplace=True)

In [7]:
data.head()

,company,city,key_skills,salary_from,salary_to,experience,schedule,salary,area,Vacancy type
0,ЛУКОЙЛ,Москва,"Power BI, Apache Airflow, SQL",150000,150000,От 1 года до 3 лет,Полный день,150000,Мск_МО,BI-аналитик
1,Всм,Москва,"Контроль и анализ ценообразования, Unit-эконом...",80000,120000,От 1 года до 3 лет,Удаленная работа,100000,Мск_МО,Финансовый аналитик
2,ИТРУМ,Ростов-на-Дону,"Системный аналитик, UML, SQL, Функциональные т...",70000,80000,Нет опыта,Удаленная работа,75000,ЮФО,Системный аналитик
3,Uniel,Москва,"Аналитическое мышление, Аналитика маркетплейсо...",80000,80000,От 1 года до 3 лет,Полный день,80000,Мск_МО,Аналитик маркетплейсов
4,ЮДжейПи,Екатеринбург,"Телефонные переговоры, Коммуникабельность, Ана...",60000,60000,От 1 года до 3 лет,Удаленная работа,60000,УФО,Другое


In [17]:
data.describe()

,salary_from,salary_to,salary,Не указано,Аналитическое мышление,MS Excel,Работа с большим объемом информации,SQL,Бизнес-анализ,Анализ данных,...,area_ЮФО,Vacancy type_1С аналитик,Vacancy type_BI-аналитик,Vacancy type_Аналитик маркетплейсов,Vacancy type_Бизнес аналитик,Vacancy type_Другое,Vacancy type_Маркетинговый аналитик,Vacancy type_Продуктовый аналитик,Vacancy type_Системный аналитик,Vacancy type_Финансовый аналитик
count,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,...,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000,1085.000000
mean,93149.196313,104770.265438,98959.730876,0.334562,0.250691,0.200000,0.145622,0.129032,0.111521,0.103226,...,0.077419,0.132719,0.028571,0.087558,0.230415,0.095853,0.120737,0.100461,0.058986,0.144700
std,40968.886541,45041.527220,41364.270166,0.472055,0.433611,0.400184,0.352890,0.335390,0.314921,0.304394,...,0.267379,0.339427,0.166675,0.282781,0.421293,0.294525,0.325972,0.300752,0.235707,0.351961
min,100.000000,20000.000000,20000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,60000.000000,70000.000000,70000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,80000.000000,100000.000000,90000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,120000.000000,137000.000000,120000.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,220000.000000,350000.000000,220000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Обучаем модели

In [10]:
# делю навыки на списки
skills = data['key_skills'].str.split(',').apply(lambda x: [s.strip() for s in x] if isinstance(x, list) else [])

# оставляю только топ 20 самых частых навыков
mlb = MultiLabelBinarizer()
skills_encoded = pd.DataFrame(mlb.fit_transform(skills), columns=mlb.classes_, index=data.index)
top_skills = skills_encoded.sum().sort_values(ascending=False).head(20).index
skills_encoded = skills_encoded[top_skills]
data = pd.concat([data, skills_encoded], axis=1)
data.drop(columns=['key_skills'], inplace=True)

In [13]:
# список категориальных
cat_features = ['company', 'city', 'experience', 'schedule', 'area', 'Vacancy type']

# encoding
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', max_categories=20)
encoded_features = encoder.fit_transform(data[cat_features])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(cat_features))
data = pd.concat([data.drop(columns=cat_features), encoded_df], axis=1)

In [18]:
data.drop(columns=['salary_from', 'salary_to'], inplace=True)

In [20]:
X = data.drop(columns=['salary'])
y = data['salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [116]:
# функция, чтоб был симпатичный вывод метрик
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

def print_metrics(y_true_train, y_pred_train, y_true_test, y_pred_test, model_name):
    print(f"{model_name}:")
    print(f"{'Metric':<10}{'Train':<15}{'Test':<15}")
    print("-" * 35)
    
    # R²
    train_r2 = r2_score(y_true_train, y_pred_train)
    test_r2 = r2_score(y_true_test, y_pred_test)
    print(f"{'R²':<10}{train_r2:.3f}{'':<8}{test_r2:.3f}")
    
    # MAE
    train_mae = mean_absolute_error(y_true_train, y_pred_train)
    test_mae = mean_absolute_error(y_true_test, y_pred_test)
    print(f"{'MAE':<10}{train_mae:.2f}{'':<8}{test_mae:.2f}")
    
    # MSE
    train_mse = mean_squared_error(y_true_train, y_pred_train)
    test_mse = mean_squared_error(y_true_test, y_pred_test)
    print(f"{'MSE':<10}{train_mse:.2f}{'':<8}{test_mse:.2f}")

Линейная регрессия

In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_train_pred_lr = model_lr.predict(X_train)
y_test_pred_lr = model_lr.predict(X_test)
print_metrics(y_train, y_train_pred_lr, y_test, y_test_pred_lr, "Linear Regression")

Linear Regression:
Metric    Train          Test           
-----------------------------------
R²        0.510        0.381
MAE       22093        24927
MSE       836233210        1063332434


Может, взаимосвязи слишком сложные и простая линейная регрессия тут не сработает.
Средняя ошибка предсказания +- 25 тыс.

Случайный лес

In [39]:
from sklearn.ensemble import RandomForestRegressor

model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
y_train_pred_rf = model_rf.predict(X_train)
y_test_pred_rf = model_rf.predict(X_test)
print_metrics(y_train, y_train_pred_rf, y_test, y_test_pred_rf, "Random Forest")

Random Forest:
Metric    Train          Test           
-----------------------------------
R²        0.844        0.322
MAE       11406        24957
MSE       266422844        1165749976


Худшая обобщающая способность среди всех моделей + тут явное переобучение (разрыв между train/test > 0.5).
Может, тут слишком глубокие деревья или избыток признаков, можно попробовать уменьшить max_depth, увеличить min_samples_leaf или отобрать только ключевые признки.

Градиентный бустинг (XGBoost)

In [40]:
from xgboost import XGBRegressor

model_xgb = XGBRegressor(n_estimators=100, random_state=42)
model_xgb.fit(X_train, y_train)
y_train_pred_xgb = model_xgb.predict(X_train)
y_test_pred_xgb = model_xgb.predict(X_test)
print_metrics(y_train, y_train_pred_xgb, y_test, y_test_pred_xgb, "XGBoost")

XGBoost:
Metric    Train          Test           
-----------------------------------
R²        0.843        0.345
MAE       10390        24190
MSE       267834800        1124840320


Относительно других двух моделей тут лучший результат по R² на тесте, но на самом деле все еще довольно низкий. Ошибка тоже меньше, чем у других моделей.
Переобучение умеренность, тк разрыв R² train/test ≈ 0.5. Это тоже лучше, чем у двух других моделей. Тоже можно попробовать поменять параметры.

# Пытаемся получить результаты лучше

In [129]:
data = pd.read_excel('all_vacancies_clean.xlsx')
data.drop(columns=['id', 'name', 'alternate_url'], inplace=True)

# логарифмируем целевую переменную
data['salary_log'] = np.log1p(data['salary'])

data.drop(columns=['salary_from', 'salary_to'], inplace=True)

In [130]:
top_skills = data['key_skills'].str.split(',').explode().str.strip().value_counts().head(20).index
for skill in top_skills:
    data[f'skill_{skill}'] = data['key_skills'].str.contains(skill).astype(int)
data.drop(columns=['key_skills'], inplace=True)

In [131]:
data.head()

,company,city,experience,schedule,salary,area,Vacancy type,salary_log,skill_Не указано,skill_Аналитическое мышление,...,skill_Формирование аналитической отчетности,skill_MS PowerPoint,skill_BPMN,skill_Аналитика,skill_Анализ бизнес-процессов,skill_Системный анализ,skill_Аналитические исследования,skill_Работа с базами данных,skill_Аналитический склад ума,skill_Работа в команде
0,ЛУКОЙЛ,Москва,От 1 года до 3 лет,Полный день,150000,Мск_МО,BI-аналитик,11.918397,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Всм,Москва,От 1 года до 3 лет,Удаленная работа,100000,Мск_МО,Финансовый аналитик,11.512935,0,0,...,0,0,0,1,0,0,0,0,0,0
2,ИТРУМ,Ростов-на-Дону,Нет опыта,Удаленная работа,75000,ЮФО,Системный аналитик,11.225257,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Uniel,Москва,От 1 года до 3 лет,Полный день,80000,Мск_МО,Аналитик маркетплейсов,11.289794,0,1,...,0,0,0,1,0,0,0,0,0,0
4,ЮДжейПи,Екатеринбург,От 1 года до 3 лет,Удаленная работа,60000,УФО,Другое,11.002117,0,1,...,0,0,0,0,0,0,0,0,0,0


In [132]:
X = data.drop(columns=['salary', 'salary_log'])
y = data['salary_log']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [133]:
cat_features = ['company', 'city', 'experience', 'schedule', 'area', 'Vacancy type']

LassoCV

In [134]:
preprocessor_lasso = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore', max_categories=10), cat_features)
])

lasso_pipe = Pipeline([
    ('prep', preprocessor_lasso),
    ('model', LassoCV(alphas=np.logspace(-3, 3, 100), cv=5, max_iter=10000))
])

lasso_pipe.fit(X_train, y_train)
y_train_pred_lp = lasso_pipe.predict(X_train)
y_test_pred_lp = lasso_pipe.predict(X_test)
print_metrics(y_train, y_train_pred_lp, y_test, y_test_pred_lp, "LassoCV")

LassoCV:
Metric    Train          Test           
-----------------------------------
R²        0.453        0.414
MAE       0.24        0.26
MSE       0.10        0.11


LassoCV показывает на тестовых данных результат лучше, чем Linear Regression, то есть регуляризация помогает нам избежать переобучения.

CatBoost

In [135]:
catboost_model = CatBoostRegressor(
    cat_features=cat_features,
    verbose=0,
    random_state=42,
    n_estimators=1000
)

catboost_model.fit(X_train, y_train)
y_train_pred_cat = catboost_model.predict(X_train)
y_test_pred_cat = catboost_model.predict(X_test)
print_metrics(y_train, y_train_pred_cat, y_test, y_test_pred_cat, "LassoCV")

LassoCV:
Metric    Train          Test           
-----------------------------------
R²        0.684        0.494
MAE       0.18        0.24
MSE       0.06        0.09


Попробуем еще прошлые модели, но на логарифмированной целевой переменной

In [136]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', max_categories=20)
encoded_features = encoder.fit_transform(data[cat_features])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(cat_features))
data = pd.concat([data.drop(columns=cat_features), encoded_df], axis=1)

In [145]:
X = data.drop(columns=['salary', 'salary_log'])
y = data['salary_log']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [146]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_train_pred_lr = model_lr.predict(X_train)
y_test_pred_lr = model_lr.predict(X_test)
print_metrics(y_train, y_train_pred_lr, y_test, y_test_pred_lr, "Linear Regression")

model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
y_train_pred_rf = model_rf.predict(X_train)
y_test_pred_rf = model_rf.predict(X_test)
print('\n')
print_metrics(y_train, y_train_pred_rf, y_test, y_test_pred_rf, "Random Forest")

model_xgb = XGBRegressor(n_estimators=100, random_state=42)
model_xgb.fit(X_train, y_train)
y_train_pred_xgb = model_xgb.predict(X_train)
y_test_pred_xgb = model_xgb.predict(X_test)
print('\n')
print_metrics(y_train, y_train_pred_xgb, y_test, y_test_pred_xgb, "XGBoost")

Linear Regression:
Metric    Train          Test           
-----------------------------------
R²        0.523        0.416
MAE       0.23        0.25
MSE       0.09        0.11


Random Forest:
Metric    Train          Test           
-----------------------------------
R²        0.853        0.384
MAE       0.12        0.25
MSE       0.03        0.11


XGBoost:
Metric    Train          Test           
-----------------------------------
R²        0.845        0.377
MAE       0.11        0.26
MSE       0.03        0.11


Пока из протестированных моделей лучшие результаты показывает LassoCV, у неё самый высокий R² на тесте (0.494) и наименьшее переобучение.